In [1]:
import os
import zipfile

In [2]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

In [3]:
from biobb_io.api.pdb import pdb

In [4]:
from biobb_model.model.fix_side_chain import fix_side_chain

In [5]:
# Create and launch bb
proteinFile = "../files/receptor_md.pdb"

# Create prop dict and inputs/outputs
fixed_pdb = "../files/receptor_md_fixed.pdb"

fix_side_chain(input_pdb_path=proteinFile,
             output_pdb_path=fixed_pdb)

2021-09-07 14:35:46,643 [MainThread  ] [INFO ]  check_structure -i ../files/receptor_md.pdb -o ../files/receptor_md_fixed.pdb --force_save fixside --fix ALL

2021-09-07 14:35:46,645 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:35:46,646 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure ../files/receptor_md.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, B: Protein)
 Num. residues:  590
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  5698

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Final Num. model

0

In [6]:
# Create Protein system topology
# Import module
from biobb_md.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro = 'receptor_md_fixed_pdb2gmx.gro'
output_pdb2gmx_top_zip = 'receptor_md_fixed_pdb2gmx_top.zip'
prop = {
    'force_field' : 'amber99sb-ildn',
    'water_type': 'spce'
}

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb,
        output_gro_path=output_pdb2gmx_gro,
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties=prop)

2021-09-07 14:35:46,757 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2021-09-07 14:35:46,758 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:35:50,828 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f ../files/receptor_md_fixed.pdb -o receptor_md_fixed_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2021-09-07 14:35:50,830 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:35:50,830 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b
going to rename amber99sb-ildn.ff/dna.r2b
going to rename amber99sb-ildn.ff/rna.r2b
Reading ../files/receptor_md_fixed.pdb...
Read '', 5698 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 2 chains and 0 blocks of water and 590 residues with 5698 atoms

  chain  #res #atoms
  1 'A'   477   4518  
  2 'B'   113   1180  

Reading residue database... (A

0

In [7]:
import openbabel as ob

In [8]:

obConversion = ob.OBConversion()
obConversion.SetInAndOutFormats( "pdbqt", "pdb")

obmol = ob.OBMol()
obConversion.ReadFile(obmol,"../files/ligand_vina_out_CHEMBL126.pdbqt")

obmol.CorrectForPH()
obmol.AddHydrogens()


print (obmol.NumAtoms())
print (obmol.NumBonds())

outMDL = obConversion.WriteFile(obmol,"../files/ligand_vina_out_CHEMBL126.pdb")

44
46


In [9]:
# Create Ligand system topology, STEP 1
# Reduce_add_hydrogens: add Hydrogen atoms to a small molecule (using Reduce tool from Ambertools package)
# Import module
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens



In [10]:
# Create prop dict and inputs/outputs
### my ligands are already minized but comes with a lot of models 
ligandFile = "../files/ligand_vina_out_CHEMBL126.pdb"
output_reduce_h = "../files/ligand_vina_out_CHEMBL126.reduce.H.pdb" 
prop = {
    'nuclear' : 'true'
}

# Create and launch bb
reduce_add_hydrogens(input_path=ligandFile,
                   output_path=output_reduce_h,
                   properties=prop)

2021-09-07 14:35:51,333 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:35:51,651 [MainThread  ] [INFO ]  reduce -NUClear -OH -ROTNH3 -ALLALT ../files/ligand_vina_out_CHEMBL126.pdb > ../files/ligand_vina_out_CHEMBL126.reduce.H.pdb

2021-09-07 14:35:51,652 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:35:51,653 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "../files/ligand_vina_out_CHEMBL126.pdb"
Database of HETATM connections: "/Users/barradd/anaconda3/envs/biobb_Protein-Complex_MDsetup_tutorial//dat/reduce_wwPDB_het_dict.txt                                                                                                                                                                                           "
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occup

0

In [11]:
# Create Ligand system topology, STEP 2
# Babel_minimize: Structure energy minimization of a small molecule after being modified adding hydrogen atoms
# Import module
from biobb_chemistry.babelm.babel_minimize import babel_minimize

# Create prop dict and inputs/outputs
output_babel_min = "../files/ligand_vina_out_CHEMBL126.reduce.H.min.mol2"                              
prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}


# Create and launch bb
babel_minimize(input_path=output_reduce_h,
              output_path=output_babel_min,
              properties=prop)

2021-09-07 14:35:51,669 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2021-09-07 14:35:51,671 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2021-09-07 14:35:51,672 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2021-09-07 14:35:51,674 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2021-09-07 14:35:51,675 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2021-09-07 14:35:51,676 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2021-09-07 14:35:51,677 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:35:53,322 [MainThread  ] [INFO ]  obminimize -c 1e-10 -sd -ff GAFF -ipdb ../files/ligand_vina_out_CHEMBL126.reduce.H.pdb -omol2 > ../files/ligand_vina_out_CHEMBL126.reduce.H.min.mol2

2021-09-07 14:35:53,323 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:35:53,324 [MainThread  ] [INFO ]  
A T O M   T Y

0

In [12]:
# Create Ligand system topology, STEP 3
# Acpype_params_gmx: Generation of topologies for GROMACS with ACPype
# Import module
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx



In [13]:
# Create prop dict and inputs/outputs
mol_charge = 0


ligandCode = "CHEMBL126"
output_acpype_gro = ligandCode+'_params.gro'
output_acpype_itp = ligandCode+'_params.itp'
output_acpype_top = ligandCode+'_params.top'
output_acpype = ligandCode+'_params'
prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

# Create and launch bb
acpype_params_gmx(input_path=output_babel_min, 
# acpype_params_gmx(input_path="../files/ligand_vina_out_CHEMBL126.mol2", 

                output_path_gro=output_acpype_gro,
                output_path_itp=output_acpype_itp,
                output_path_top=output_acpype_top,
                properties=prop)

2021-09-07 14:35:53,357 [MainThread  ] [INFO ]  Running acpype, this execution can take a while
2021-09-07 14:35:53,358 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:36:53,057 [MainThread  ] [INFO ]  acpype -i /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks/../files/ligand_vina_out_CHEMBL126.reduce.H.min.mol2 -b CHEMBL126_params.4OECNA -n 0

2021-09-07 14:36:53,059 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:36:53,060 [MainThread  ] [INFO ]  ========================================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2019-11-07T23:16:00CET (c) 2021 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'LIG'....
Checking parameters for unit 'LIG'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_

0

In [14]:
# MakeNdx: Creating index file with a new group (small molecule heavy atoms)
from biobb_md.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_ligand_ndx = ligandCode+'_index.ndx'
prop = {
    'selection': "0 & ! a H*"
}

# Create and launch bb
make_ndx(input_structure_path=output_acpype_gro,
        output_ndx_path=output_ligand_ndx,
        properties=prop)

2021-09-07 14:36:53,151 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2021-09-07 14:36:53,153 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:36:53,197 [MainThread  ] [INFO ]  echo -e '0 & ! a H*\nq' | gmx -nobackup -nocopyright make_ndx -f CHEMBL126_params.gro -o CHEMBL126_index.ndx

2021-09-07 14:36:53,199 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:36:53,200 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :    44 atoms
  1 Other               :    44 atoms
  2 LIG                 :    44 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'c

0

In [15]:
# Genrestr: Generating the position restraints file
from biobb_md.gromacs.genrestr import genrestr

# Create prop dict and inputs/outputs
output_restraints_top = ligandCode+'_posres.itp'
prop = {
    'force_constants': "1000 1000 1000",
    'restrained_group': "System"
}

# Create and launch bb
genrestr(input_structure_path=output_acpype_gro,
         input_ndx_path=output_ligand_ndx,
         output_itp_path=output_restraints_top,
         properties=prop)

2021-09-07 14:36:53,267 [MainThread  ] [INFO ]  GROMACS Genrestr 20191 version detected
2021-09-07 14:36:53,270 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:36:53,330 [MainThread  ] [INFO ]  echo "System" | gmx -nobackup -nocopyright genrestr -f CHEMBL126_params.gro -o CHEMBL126_posres.itp -fc 1000 1000 1000 -n CHEMBL126_index.ndx

2021-09-07 14:36:53,332 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:36:53,334 [MainThread  ] [INFO ]  Select group to position restrain
Selected 0: 'System'

2021-09-07 14:36:53,334 [MainThread  ] [INFO ]                       :-) GROMACS - gmx genrestr, 2019.1 (-:

Executable:   /Users/barradd/anaconda3/envs/biobb_Protein-Complex_MDsetup_tutorial/bin/gmx
Data prefix:  /Users/barradd/anaconda3/envs/biobb_Protein-Complex_MDsetup_tutorial
Working dir:  /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks
Command line:
  gmx -nobackup -nocopyright genrestr -f CHEMBL126_params.gro -o CHEMBL126_posres.itp -fc 1000 1000 1000 -n CHEMB

0

In [16]:
# biobb analysis module
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Convert gro (with hydrogens) to pdb (PROTEIN)
pdbCode = "SDHA"

proteinFile_H = pdbCode+'_'+ligandCode+'_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_pdb2gmx_gro,
              input_top_path=output_pdb2gmx_gro,
              output_str_path=proteinFile_H, 
              properties=prop)

# Convert gro (with hydrogens) to pdb (LIGAND)
ligandFile_H = ligandCode+'_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_acpype_gro,
              input_top_path=output_acpype_gro,
              output_str_path=ligandFile_H, 
              properties=prop)


# Concatenating both PDB files: Protein + Ligand
complexFile_H = pdbCode+'_'+ligandCode+'_H.pdb'

# Create and launch bb
cat_pdb(input_structure1=proteinFile_H,
       input_structure2=ligandFile_H,
       output_structure_path=complexFile_H)

2021-09-07 14:36:53,363 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:36:53,501 [MainThread  ] [INFO ]  echo "System" | gmx trjconv -f /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks/receptor_md_fixed_pdb2gmx.gro -s /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks/receptor_md_fixed_pdb2gmx.gro -o SDHA_CHEMBL126_complex_H.pdb

2021-09-07 14:36:53,503 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:36:53,504 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 0: 'System'

2021-09-07 14:36:53,505 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gra

0

In [17]:
# AppendLigand: Append a ligand to a GROMACS topology
# Import module
from biobb_md.gromacs_extra.append_ligand import append_ligand

# Create prop dict and inputs/outputs
output_complex_top = pdbCode+'_'+ligandCode+'_complex.top.zip'

posresifdef = "POSRES_"+ligandCode.upper()
prop = {
    'posres_name': posresifdef
}

# Create and launch bb
append_ligand(input_top_zip_path=output_pdb2gmx_top_zip,
             input_posres_itp_path=output_restraints_top,
             input_itp_path=output_acpype_itp, 
             output_top_zip_path=output_complex_top,
             properties=prop)

2021-09-07 14:36:53,629 [MainThread  ] [INFO ]  Extracting: /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks/receptor_md_fixed_pdb2gmx_top.zip
2021-09-07 14:36:53,630 [MainThread  ] [INFO ]  to:
2021-09-07 14:36:53,630 [MainThread  ] [INFO ]  ['2a7f360a-2184-46a0-97fd-590d3f29d879/p2g.top', '2a7f360a-2184-46a0-97fd-590d3f29d879/p2g_Protein_chain_A.itp', '2a7f360a-2184-46a0-97fd-590d3f29d879/p2g_Protein_chain_B.itp', '2a7f360a-2184-46a0-97fd-590d3f29d879/posre_Protein_chain_A.itp', '2a7f360a-2184-46a0-97fd-590d3f29d879/posre_Protein_chain_B.itp']
2021-09-07 14:36:53,631 [MainThread  ] [INFO ]  Unzipping: 
2021-09-07 14:36:53,632 [MainThread  ] [INFO ]  receptor_md_fixed_pdb2gmx_top.zip
2021-09-07 14:36:53,632 [MainThread  ] [INFO ]  To: 
2021-09-07 14:36:53,633 [MainThread  ] [INFO ]  2a7f360a-2184-46a0-97fd-590d3f29d879/p2g.top
2021-09-07 14:36:53,634 [MainThread  ] [INFO ]  2a7f360a-2184-46a0-97fd-590d3f29d879/p2g_Protein_chain_A.itp
2021-09-07 14:36:53,635 [MainThread  ] [

0

In [19]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = pdbCode+'_'+ligandCode+'_complex_editconf.gro'

prop = {
    'box_type': 'octahedron',
    'distance_to_molecule': 1.0,
    'center_molecule': True
}

# Create and launch bb
editconf(input_gro_path=complexFile_H, 
         output_gro_path=output_editconf_gro,
         properties=prop)

2021-09-07 14:37:59,890 [MainThread  ] [INFO ]  GROMACS Editconf 20191 version detected
2021-09-07 14:37:59,891 [MainThread  ] [INFO ]  Centering molecule in the box.
2021-09-07 14:37:59,892 [MainThread  ] [INFO ]  Distance of the box to molecule:   1.00
2021-09-07 14:37:59,893 [MainThread  ] [INFO ]  Box type: octahedron
2021-09-07 14:37:59,893 [MainThread  ] [INFO ]  Not using any container
2021-09-07 14:38:00,097 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f SDHA_CHEMBL126_H.pdb -o SDHA_CHEMBL126_complex_editconf.gro -d 1.0 -bt octahedron -c

2021-09-07 14:38:00,098 [MainThread  ] [INFO ]  Exit code 0

2021-09-07 14:38:00,099 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 9293 atoms
Volume: 6140.98 nm^3, corresponds to roughly 2763400 electrons
No velocities found
    system size :  7.682  5.598  8.026 (nm)
    diameter    :  8.497               (nm)
    center      :  2.690  3.007  5.223 (nm

0

In [20]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro = pdbCode+'_'+ligandCode+'_solvate.gro'
output_solvate_top_zip = pdbCode+'_'+ligandCode+'_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro,
        output_gro_path=output_solvate_gro,
        input_top_zip_path=output_complex_top,
        output_top_zip_path=output_solvate_top_zip)

2021-09-07 14:38:02,934 [MainThread  ] [INFO ]  GROMACS Solvate 20191 version detected
2021-09-07 14:38:02,956 [MainThread  ] [INFO ]  Extracting: /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks/SDHA_CHEMBL126_complex.top.zip
2021-09-07 14:38:02,957 [MainThread  ] [INFO ]  to:
2021-09-07 14:38:02,958 [MainThread  ] [INFO ]  ['362e864f-0eef-45b4-8e5b-2f9e5c83f40c/CHEMBL126_params.itp', '362e864f-0eef-45b4-8e5b-2f9e5c83f40c/CHEMBL126_posres.itp', '362e864f-0eef-45b4-8e5b-2f9e5c83f40c/ligand.top', '362e864f-0eef-45b4-8e5b-2f9e5c83f40c/p2g_Protein_chain_A.itp', '362e864f-0eef-45b4-8e5b-2f9e5c83f40c/p2g_Protein_chain_B.itp', '362e864f-0eef-45b4-8e5b-2f9e5c83f40c/posre_Protein_chain_A.itp', '362e864f-0eef-45b4-8e5b-2f9e5c83f40c/posre_Protein_chain_B.itp']
2021-09-07 14:38:02,959 [MainThread  ] [INFO ]  Unzipping: 
2021-09-07 14:38:02,959 [MainThread  ] [INFO ]  SDHA_CHEMBL126_complex.top.zip
2021-09-07 14:38:02,960 [MainThread  ] [INFO ]  To: 
2021-09-07 14:38:02,961 [MainThread 

0

In [21]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000'
    },
    'simulation_type':'minimization',
    'maxwarn': 1
}
output_gppion_tpr = pdbCode+'_'+ligandCode+'_complex_gppion.tpr'

# Create and launch bb
grompp(input_gro_path=output_solvate_gro,
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,
       properties=prop)

2021-09-07 14:38:03,822 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2021-09-07 14:38:03,846 [MainThread  ] [INFO ]  Extracting: /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks/SDHA_CHEMBL126_solvate_top.zip
2021-09-07 14:38:03,847 [MainThread  ] [INFO ]  to:
2021-09-07 14:38:03,848 [MainThread  ] [INFO ]  ['222cd29a-27e5-48a8-b5af-54cd596607fd/CHEMBL126_params.itp', '222cd29a-27e5-48a8-b5af-54cd596607fd/CHEMBL126_posres.itp', '222cd29a-27e5-48a8-b5af-54cd596607fd/ligand.top', '222cd29a-27e5-48a8-b5af-54cd596607fd/p2g_Protein_chain_A.itp', '222cd29a-27e5-48a8-b5af-54cd596607fd/p2g_Protein_chain_B.itp', '222cd29a-27e5-48a8-b5af-54cd596607fd/posre_Protein_chain_A.itp', '222cd29a-27e5-48a8-b5af-54cd596607fd/posre_Protein_chain_B.itp']
2021-09-07 14:38:03,849 [MainThread  ] [INFO ]  Unzipping: 
2021-09-07 14:38:03,850 [MainThread  ] [INFO ]  SDHA_CHEMBL126_solvate_top.zip
2021-09-07 14:38:03,851 [MainThread  ] [INFO ]  To: 
2021-09-07 14:38:03,851 [MainThread  

0

In [22]:
# Genion: Adding ions to reach a 0.05 molar concentration
from biobb_md.gromacs.genion import genion

# Create prop dict and inputs/outputs
prop={
    'neutral':True,
    'concentration':0.05
}
output_genion_gro = pdbCode+'_'+ligandCode+'_genion.gro'
output_genion_top_zip = pdbCode+'_'+ligandCode+'_genion_top.zip'

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr,
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip,
       output_top_zip_path=output_genion_top_zip, 
       properties=prop)

2021-09-07 14:38:08,454 [MainThread  ] [INFO ]  GROMACS Genion 20191 version detected
2021-09-07 14:38:08,477 [MainThread  ] [INFO ]  Extracting: /Users/barradd/Documents/BARRADD_Things/SDHA_CADD/notebooks/SDHA_CHEMBL126_solvate_top.zip
2021-09-07 14:38:08,478 [MainThread  ] [INFO ]  to:
2021-09-07 14:38:08,480 [MainThread  ] [INFO ]  ['e1506be4-e440-4371-86c8-149516dc397f/CHEMBL126_params.itp', 'e1506be4-e440-4371-86c8-149516dc397f/CHEMBL126_posres.itp', 'e1506be4-e440-4371-86c8-149516dc397f/ligand.top', 'e1506be4-e440-4371-86c8-149516dc397f/p2g_Protein_chain_A.itp', 'e1506be4-e440-4371-86c8-149516dc397f/p2g_Protein_chain_B.itp', 'e1506be4-e440-4371-86c8-149516dc397f/posre_Protein_chain_A.itp', 'e1506be4-e440-4371-86c8-149516dc397f/posre_Protein_chain_B.itp']
2021-09-07 14:38:08,481 [MainThread  ] [INFO ]  Unzipping: 
2021-09-07 14:38:08,482 [MainThread  ] [INFO ]  SDHA_CHEMBL126_solvate_top.zip
2021-09-07 14:38:08,483 [MainThread  ] [INFO ]  To: 
2021-09-07 14:38:08,484 [MainThread  

0

In [23]:
# import nglview
# import ipywidgets

In [24]:
# #Show protein
# view = nglview.show_structure_file(output_genion_gro)
# view.clear_representations()
# view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
# view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
# view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')
# view.add_representation(repr_type='ball+stick', selection='NA')
# view.add_representation(repr_type='ball+stick', selection='CL')
# view._remote_call('setSize', target='Widget', args=['','600px'])
# view.camera='orthographic'

# view.render_image()
# view.download_image(filename=f'../files/{pdbCode}_{ligandCode}_genion.png')

# view

In [25]:
### gather thisn in the apropiate folder 


In [26]:
# gather_files = [ x for x in os.listdir(".") if ligandCode in x ]

In [27]:
def clean_WS(): 
    os.system(f"mkdir -p ../files/{ligandCode}")
    gather_files = [ x for x in os.listdir(".") if ligandCode in x ]
    erase_log_files = [ x for x in os.listdir(".") if "log" == x[0:3]]
    erase_itp = [ x for x in os.listdir(".") if ".itp" == x[-4:]]
    erase_pdb2gmx = [ x for x in os.listdir(".") if "pdb2gmx" in x]
    for x in gather_files:
        os.system(f"mv {x} ../files/{ligandCode}/")
    for x in erase_log_files:
        if os.path.isfile(x) : 
            os.remove(x)
            
    for x in erase_itp :
         if os.path.isfile(x) : 
            os.remove(x)
            
    for x in erase_pdb2gmx :
         if os.path.isfile(x) : 
            os.remove(x)
    
    print (len (gather_files))
    print (len (erase_log_files))
    print (len (erase_itp))
    print (len (erase_pdb2gmx))

In [28]:
clean_WS()

15
30
6
2


# At this poitn I could proceed with gromacs protocol 